<a href="https://colab.research.google.com/github/bhanuteja2001/Dphi_DL/blob/master/DLBAS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dphi-official/Datasets.git

Cloning into 'Datasets'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 9874 (delta 8), reused 17 (delta 2), pack-reused 9847
Receiving objects: 100% (9874/9874), 427.70 MiB | 33.09 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Checking out files: 100% (9345/9345), done.


In [2]:
!cp -r Datasets/animal_data /content/animal_data/

In [3]:
# unzipping the contents
!unzip animal_data/train_beg.zip

Archive:  animal_data/train_beg.zip
   creating: train_beg/
  inflating: train_beg/Image_1.jpg   
  inflating: train_beg/Image_10.jpg  
  inflating: train_beg/Image_100.jpg  
  inflating: train_beg/Image_1000.jpg  
  inflating: train_beg/Image_1001.jpg  
  inflating: train_beg/Image_1002.jpg  
  inflating: train_beg/Image_1003.jpg  
  inflating: train_beg/Image_1004.jpg  
  inflating: train_beg/Image_1005.jpg  
  inflating: train_beg/Image_1006.jpg  
  inflating: train_beg/Image_1007.jpg  
  inflating: train_beg/Image_1008.jpg  
  inflating: train_beg/Image_1009.jpg  
  inflating: train_beg/Image_101.jpg  
  inflating: train_beg/Image_1010.jpg  
  inflating: train_beg/Image_1011.jpg  
  inflating: train_beg/Image_1012.jpg  
  inflating: train_beg/Image_1013.jpg  
  inflating: train_beg/Image_1014.jpg  
  inflating: train_beg/Image_1015.jpg  
  inflating: train_beg/Image_1016.jpg  
  inflating: train_beg/Image_1017.jpg  
  inflating: train_beg/Image_1018.jpg  
  inflating: train_beg/Ima

In [4]:
import numpy as np
import pandas as pd

In [5]:
Training_data = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/animal_data/Training_set_animals.csv")

In [6]:
# getting the path of all image files
filepaths = ['/content/train_beg/' + fname[:-4] + '.jpg' for fname in Training_data['filename']]
filepaths

['/content/train_beg/Image_1.jpg',
 '/content/train_beg/Image_2.jpg',
 '/content/train_beg/Image_3.jpg',
 '/content/train_beg/Image_4.jpg',
 '/content/train_beg/Image_5.jpg',
 '/content/train_beg/Image_6.jpg',
 '/content/train_beg/Image_7.jpg',
 '/content/train_beg/Image_8.jpg',
 '/content/train_beg/Image_9.jpg',
 '/content/train_beg/Image_10.jpg',
 '/content/train_beg/Image_11.jpg',
 '/content/train_beg/Image_12.jpg',
 '/content/train_beg/Image_13.jpg',
 '/content/train_beg/Image_14.jpg',
 '/content/train_beg/Image_15.jpg',
 '/content/train_beg/Image_16.jpg',
 '/content/train_beg/Image_17.jpg',
 '/content/train_beg/Image_18.jpg',
 '/content/train_beg/Image_19.jpg',
 '/content/train_beg/Image_20.jpg',
 '/content/train_beg/Image_21.jpg',
 '/content/train_beg/Image_22.jpg',
 '/content/train_beg/Image_23.jpg',
 '/content/train_beg/Image_24.jpg',
 '/content/train_beg/Image_25.jpg',
 '/content/train_beg/Image_26.jpg',
 '/content/train_beg/Image_27.jpg',
 '/content/train_beg/Image_28.jpg',
 

In [7]:
Training_data.tail()

filename animal_type
1195  Image_1196.jpg      pecora
1196  Image_1197.jpg      pecora
1197  Image_1198.jpg      pecora
1198  Image_1199.jpg      pecora
1199  Image_1200.jpg      pecora

In [8]:
Training_data.animal_type.value_counts()

pecora    600
mucca     600
Name: animal_type, dtype: int64

In [9]:
# Confirm if number of images is same as number of labels given
if len(Training_data) == len(filepaths):
    print('Number of labels i.e. ', len(Training_data), 'matches the number of filenames i.e. ', len(filepaths))
else:
    print('Number of labels doesnot matches the number of filenames')

Number of labels i.e.  1200 matches the number of filenames i.e.  1200


In [10]:
images = [[fname, '/content/train_beg/' + fname[:-4] + '.jpg'] for fname in Training_data['filename']]
images = pd.DataFrame(images)
images.columns = ['filename', 'file']
images

filename                               file
0        Image_1.jpg     /content/train_beg/Image_1.jpg
1        Image_2.jpg     /content/train_beg/Image_2.jpg
2        Image_3.jpg     /content/train_beg/Image_3.jpg
3        Image_4.jpg     /content/train_beg/Image_4.jpg
4        Image_5.jpg     /content/train_beg/Image_5.jpg
...              ...                                ...
1195  Image_1196.jpg  /content/train_beg/Image_1196.jpg
1196  Image_1197.jpg  /content/train_beg/Image_1197.jpg
1197  Image_1198.jpg  /content/train_beg/Image_1198.jpg
1198  Image_1199.jpg  /content/train_beg/Image_1199.jpg
1199  Image_1200.jpg  /content/train_beg/Image_1200.jpg

[1200 rows x 2 columns]

In [11]:

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
%load_ext tensorboard

In [16]:
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [14]:
import random
import matplotlib.pyplot as plt
import datetime

In [13]:
print(Training_data.shape)
Training_data.head()

(1200, 2)


filename animal_type
0  Image_1.jpg       mucca
1  Image_2.jpg       mucca
2  Image_3.jpg       mucca
3  Image_4.jpg       mucca
4  Image_5.jpg       mucca

In [12]:
print(images.shape)
images.head()


(1200, 2)


filename                            file
0  Image_1.jpg  /content/train_beg/Image_1.jpg
1  Image_2.jpg  /content/train_beg/Image_2.jpg
2  Image_3.jpg  /content/train_beg/Image_3.jpg
3  Image_4.jpg  /content/train_beg/Image_4.jpg
4  Image_5.jpg  /content/train_beg/Image_5.jpg

In [18]:
Train = pd.merge(Training_data, images, on='filename')
Train.drop(columns = ['filename'], inplace = True)

In [22]:
Train.head()

animal_type                            file
0       mucca  /content/train_beg/Image_1.jpg
1       mucca  /content/train_beg/Image_2.jpg
2       mucca  /content/train_beg/Image_3.jpg
3       mucca  /content/train_beg/Image_4.jpg
4       mucca  /content/train_beg/Image_5.jpg

In [24]:
train_image = []
for i in tqdm(range(Train.shape[0])):
    img = image.load_img(Train['file'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 1200/1200 [00:02<00:00, 583.50it/s]


In [35]:
y = Train['animal_type'].values
from sklearn.preprocessing import LabelEncoder
import numpy as np

y = np.array(y)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [86]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [92]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [93]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [94]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
30/30 [==============================] - 2s 82ms/step - loss: 0.6952 - accuracy: 0.5312 - val_loss: 0.6801 - val_accuracy: 0.6708
Epoch 2/10
30/30 [==============================] - 2s 76ms/step - loss: 0.6562 - accuracy: 0.6302 - val_loss: 0.6131 - val_accuracy: 0.7000
Epoch 3/10
30/30 [==============================] - 2s 76ms/step - loss: 0.6148 - accuracy: 0.6833 - val_loss: 0.5821 - val_accuracy: 0.7000
Epoch 4/10
30/30 [==============================] - 2s 76ms/step - loss: 0.5724 - accuracy: 0.7063 - val_loss: 0.5659 - val_accuracy: 0.6875
Epoch 5/10
30/30 [==============================] - 2s 77ms/step - loss: 0.5494 - accuracy: 0.7323 - val_loss: 0.5621 - val_accuracy: 0.7000
Epoch 6/10
30/30 [==============================] - 2s 76ms/step - loss: 0.4784 - accuracy: 0.7833 - val_loss: 0.5595 - val_accuracy: 0.7042
Epoch 7/10
30/30 [==============================] - 2s 77ms/step - loss: 0.4532 - accuracy: 0.7990 - val_loss: 0.5452 - val_accuracy: 0.7083
Epoch 8/10
30

In [91]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 26, 26, 960)       26880     
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 24, 24, 64)        553024    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_17 (Dropout)         (None, 128)              

In [95]:
Testing_set_animals = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/animal_data/Testing_set_animals.csv")

In [108]:
Testing_set_animals.columns = ['filename']

In [112]:
new_row = pd.DataFrame({'filename':'Image_1.jpg'},index =[0])
Testing_set_animals = pd.concat([new_row, Testing_set_animals]).reset_index(drop = True) 
Testing_set_animals.head(5)

filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg

In [113]:
Testing_set_animals.head()

filename
0  Image_1.jpg
1  Image_2.jpg
2  Image_3.jpg
3  Image_4.jpg
4  Image_5.jpg

In [97]:
!unzip animal_data/test_beg.zip

Archive:  animal_data/test_beg.zip
   creating: test_beg/
  inflating: test_beg/Image_1.jpg    
  inflating: test_beg/Image_10.jpg   
  inflating: test_beg/Image_100.jpg  
  inflating: test_beg/Image_101.jpg  
  inflating: test_beg/Image_102.jpg  
  inflating: test_beg/Image_103.jpg  
  inflating: test_beg/Image_104.jpg  
  inflating: test_beg/Image_105.jpg  
  inflating: test_beg/Image_106.jpg  
  inflating: test_beg/Image_107.jpg  
  inflating: test_beg/Image_108.jpg  
  inflating: test_beg/Image_109.jpg  
  inflating: test_beg/Image_11.jpg   
  inflating: test_beg/Image_110.jpg  
  inflating: test_beg/Image_111.jpg  
  inflating: test_beg/Image_112.jpg  
  inflating: test_beg/Image_113.jpg  
  inflating: test_beg/Image_114.jpg  
  inflating: test_beg/Image_115.jpg  
  inflating: test_beg/Image_116.jpg  
  inflating: test_beg/Image_117.jpg  
  inflating: test_beg/Image_118.jpg  
  inflating: test_beg/Image_119.jpg  
  inflating: test_beg/Image_12.jpg   
  inflating: test_beg/Image_12

In [114]:
filepaths = ['/content/test_beg/' + fname[:-4] + '.jpg' for fname in Testing_set_animals['filename']]
filepaths

['/content/test_beg/Image_1.jpg',
 '/content/test_beg/Image_2.jpg',
 '/content/test_beg/Image_3.jpg',
 '/content/test_beg/Image_4.jpg',
 '/content/test_beg/Image_5.jpg',
 '/content/test_beg/Image_6.jpg',
 '/content/test_beg/Image_7.jpg',
 '/content/test_beg/Image_8.jpg',
 '/content/test_beg/Image_9.jpg',
 '/content/test_beg/Image_10.jpg',
 '/content/test_beg/Image_11.jpg',
 '/content/test_beg/Image_12.jpg',
 '/content/test_beg/Image_13.jpg',
 '/content/test_beg/Image_14.jpg',
 '/content/test_beg/Image_15.jpg',
 '/content/test_beg/Image_16.jpg',
 '/content/test_beg/Image_17.jpg',
 '/content/test_beg/Image_18.jpg',
 '/content/test_beg/Image_19.jpg',
 '/content/test_beg/Image_20.jpg',
 '/content/test_beg/Image_21.jpg',
 '/content/test_beg/Image_22.jpg',
 '/content/test_beg/Image_23.jpg',
 '/content/test_beg/Image_24.jpg',
 '/content/test_beg/Image_25.jpg',
 '/content/test_beg/Image_26.jpg',
 '/content/test_beg/Image_27.jpg',
 '/content/test_beg/Image_28.jpg',
 '/content/test_beg/Image_29.

In [116]:
Timages = [[fname, '/content/test_beg/' + fname[:-4] + '.jpg'] for fname in Testing_set_animals['filename']]
Timages = pd.DataFrame(Timages)
Timages.columns = ['filename', 'file']
Timages

filename                             file
0      Image_1.jpg    /content/test_beg/Image_1.jpg
1      Image_2.jpg    /content/test_beg/Image_2.jpg
2      Image_3.jpg    /content/test_beg/Image_3.jpg
3      Image_4.jpg    /content/test_beg/Image_4.jpg
4      Image_5.jpg    /content/test_beg/Image_5.jpg
..             ...                              ...
395  Image_396.jpg  /content/test_beg/Image_396.jpg
396  Image_397.jpg  /content/test_beg/Image_397.jpg
397  Image_398.jpg  /content/test_beg/Image_398.jpg
398  Image_399.jpg  /content/test_beg/Image_399.jpg
399  Image_400.jpg  /content/test_beg/Image_400.jpg

[400 rows x 2 columns]

In [117]:
test_image = []
for i in tqdm(range(Timages.shape[0])):
    img = image.load_img(Timages['file'][i], target_size=(28,28,1), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
X = np.array(test_image)

100%|██████████| 400/400 [00:00<00:00, 613.85it/s]


In [118]:
prediction = model.predict_classes(X)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [126]:
Ans = []
for i in prediction:
  if i == 0:
    Ans.append('mucca')
  else:
    Ans.append('pecora')

In [135]:
A = np.array(Ans).reshape(1,400)

In [136]:
A

array([['pecora', 'pecora', 'mucca', 'pecora', 'pecora', 'mucca',
        'mucca', 'pecora', 'mucca', 'pecora', 'mucca', 'pecora', 'mucca',
        'mucca', 'mucca', 'pecora', 'pecora', 'pecora', 'pecora',
        'pecora', 'mucca', 'pecora', 'mucca', 'pecora', 'pecora',
        'pecora', 'mucca', 'pecora', 'pecora', 'pecora', 'pecora',
        'pecora', 'pecora', 'pecora', 'pecora', 'mucca', 'mucca',
        'pecora', 'pecora', 'mucca', 'pecora', 'mucca', 'pecora',
        'mucca', 'pecora', 'mucca', 'pecora', 'pecora', 'pecora',
        'mucca', 'pecora', 'mucca', 'mucca', 'pecora', 'pecora', 'mucca',
        'mucca', 'pecora', 'pecora', 'pecora', 'pecora', 'pecora',
        'pecora', 'mucca', 'pecora', 'mucca', 'pecora', 'mucca',
        'pecora', 'pecora', 'pecora', 'pecora', 'mucca', 'mucca',
        'pecora', 'pecora', 'pecora', 'pecora', 'pecora', 'pecora',
        'pecora', 'mucca', 'mucca', 'pecora', 'mucca', 'mucca', 'mucca',
        'pecora', 'pecora', 'pecora', 'pecora', 'm